In [33]:
#importing the libraries
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import os
import shutil

# Set the path of the directory containing the images
dir_path = '/content/drive/MyDrive/directory_1'

# Set the paths of the two new directories for edited and unedited images
edited_dir_path = os.path.join(dir_path, '/content/drive/MyDrive/directory_1/edit_1')
unedited_dir_path = os.path.join(dir_path, '/content/drive/MyDrive/directory_1/IMAGICA_new')

# Create the new directories if they don't already exist
if not os.path.exists(edited_dir_path):
    os.makedirs(edited_dir_path)
if not os.path.exists(unedited_dir_path):
    os.makedirs(unedited_dir_path)

# Iterate over all files in the original directory
for filename in os.listdir(dir_path):
    # Check if the file is a JPG image
    if filename.endswith('.jpg'):
        # Check the last character of the file name to determine if it is edited or unedited
        if filename[-1] == '0':
            # Move the file to the unedited directory
            shutil.move(os.path.join(dir_path, filename), os.path.join(unedited_dir_path, filename))
        elif filename[-1] == '1':
            # Move the file to the edited directory
            shutil.move(os.path.join(dir_path, filename), os.path.join(edited_dir_path, filename))


In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set the paths of the edited and unedited image directories
edited_dir = '/content/drive/MyDrive/directory_1/edit_1'
unedited_dir = '/content/drive/MyDrive/directory_1/IMAGICA_new'

# Set the image dimensions and batch size
img_height = 128
img_width = 128
batch_size = 32

# Create the training and validation data generators
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    edited_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=123
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    edited_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=123
)

# Combine the training and validation data for a larger training set
train_ds = train_ds.concatenate(val_ds)

# Create the test data generator
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    unedited_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Create the CNN model
model = keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=15)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy:', accuracy)


Test accuracy: 0.8343758525459
